# Working Capital Optimization for Oil & Gas Treasury

## Use Case Overview

**Business Problem:** Oil & Gas refiners have massive working capital needs:
- **Receivables**: $500M-2B outstanding at any time
- **Payables**: Crude suppliers, contractors, utilities
- **Inventory**: Crude, refined products, spare parts
- **Cash Conversion Cycle**: 30-90 days

**Working Capital Challenges:**
- Crude payment terms: Typically 30 days
- Product receivables: 30-60 days
- Inventory holding: High value, volatile pricing
- Seasonal demand variations

**AI Applications:**
1. Cash Conversion Cycle (CCC) optimization
2. Receivables aging prediction
3. Payment timing optimization
4. Inventory turnover analysis
5. Working capital forecasting

---

## Learning Objectives

1. Calculate key working capital metrics
2. Build receivables prediction model
3. Optimize payment strategies
4. Create working capital dashboard

In [ ]:
# Imports
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

import sys
sys.path.append('..')
from src.treasury_sim.generators import set_seed

print("✅ Setup complete!")

## 1. Generate Working Capital Data

### Thinking Trace 🧠

> **Working Capital Components:**
> 
> **Current Assets:**
> - Cash and equivalents
> - Accounts receivable (AR)
> - Inventory (crude, products)
> - Prepaid expenses
> 
> **Current Liabilities:**
> - Accounts payable (AP)
> - Short-term debt
> - Accrued expenses
> 
> **Net Working Capital = Current Assets - Current Liabilities**

In [ ]:
set_seed(42)

def generate_receivables(n_invoices=500, seed=42):
    """Generate accounts receivable data"""
    np.random.seed(seed)
    
    customers = [
        ('Shell Trading', 'AAA', 0.02),
        ('BP Oil', 'AAA', 0.02),
        ('TotalEnergies', 'AA', 0.03),
        ('Repsol', 'AA', 0.04),
        ('Local Distributor A', 'BBB', 0.08),
        ('Local Distributor B', 'BBB', 0.10),
        ('Regional Trader', 'BB', 0.15),
        ('Small Dealer Network', 'B', 0.20)
    ]
    
    data = []
    base_date = datetime(2024, 1, 1)
    
    for i in range(n_invoices):
        customer, rating, late_prob = customers[np.random.choice(len(customers))]
        
        # Invoice details
        invoice_date = base_date + timedelta(days=np.random.randint(0, 365))
        
        # Amount varies by customer type
        if rating in ['AAA', 'AA']:
            amount = np.random.uniform(1_000_000, 20_000_000)
        elif rating == 'BBB':
            amount = np.random.uniform(100_000, 2_000_000)
        else:
            amount = np.random.uniform(10_000, 500_000)
        
        # Payment terms
        terms = np.random.choice([15, 30, 45, 60], p=[0.1, 0.5, 0.25, 0.15])
        due_date = invoice_date + timedelta(days=terms)
        
        # Simulate payment behavior
        is_late = np.random.random() < late_prob
        if is_late:
            days_late = np.random.randint(1, 60)
            actual_payment = due_date + timedelta(days=days_late)
        else:
            days_early = np.random.randint(0, 5)
            actual_payment = due_date - timedelta(days=days_early)
            days_late = -days_early
        
        # Status
        today = datetime(2024, 12, 15)
        if actual_payment <= today:
            status = 'PAID'
        elif due_date < today:
            status = 'OVERDUE'
            actual_payment = None
        else:
            status = 'OPEN'
            actual_payment = None
        
        data.append({
            'invoice_id': f'INV-{i+1:05d}',
            'customer': customer,
            'credit_rating': rating,
            'invoice_date': invoice_date,
            'amount': round(amount, 2),
            'payment_terms': terms,
            'due_date': due_date,
            'actual_payment_date': actual_payment,
            'days_late': days_late if status == 'PAID' else None,
            'status': status,
            'late_payment_prob': late_prob
        })
    
    return pd.DataFrame(data)

def generate_payables(n_invoices=400, seed=42):
    """Generate accounts payable data"""
    np.random.seed(seed)
    
    suppliers = [
        ('Saudi Aramco', 'Crude', 30),
        ('ADNOC', 'Crude', 30),
        ('KPC', 'Crude', 30),
        ('Maintenance Co', 'Services', 45),
        ('Utilities Provider', 'Utilities', 30),
        ('Chemical Supplier', 'Chemicals', 45),
        ('Equipment Vendor', 'Equipment', 60),
        ('Logistics Partner', 'Logistics', 30)
    ]
    
    data = []
    base_date = datetime(2024, 1, 1)
    
    for i in range(n_invoices):
        supplier, category, std_terms = suppliers[np.random.choice(len(suppliers))]
        
        invoice_date = base_date + timedelta(days=np.random.randint(0, 365))
        
        # Amount varies by category
        if category == 'Crude':
            amount = np.random.uniform(10_000_000, 100_000_000)
        elif category == 'Services':
            amount = np.random.uniform(100_000, 2_000_000)
        else:
            amount = np.random.uniform(10_000, 500_000)
        
        due_date = invoice_date + timedelta(days=std_terms)
        
        # Early payment discount
        has_discount = np.random.random() < 0.3
        discount_pct = 2.0 if has_discount else 0.0
        discount_days = 10 if has_discount else 0
        
        data.append({
            'invoice_id': f'AP-{i+1:05d}',
            'supplier': supplier,
            'category': category,
            'invoice_date': invoice_date,
            'amount': round(amount, 2),
            'payment_terms': std_terms,
            'due_date': due_date,
            'early_discount_pct': discount_pct,
            'early_discount_days': discount_days,
            'status': np.random.choice(['OPEN', 'PAID', 'SCHEDULED'], p=[0.3, 0.5, 0.2])
        })
    
    return pd.DataFrame(data)

# Generate data
receivables = generate_receivables(500, seed=42)
payables = generate_payables(400, seed=42)

print(f"📊 Generated {len(receivables):,} receivables")
print(f"📊 Generated {len(payables):,} payables")

In [ ]:
# Preview data
print("RECEIVABLES (Sample):")
display(receivables.head())

print("\nPAYABLES (Sample):")
display(payables.head())

## 2. Working Capital Metrics

### Thinking Trace 🧠

> **Key Working Capital KPIs:**
> 
> - **DSO** (Days Sales Outstanding) = (AR / Revenue) × 365
>   - How long to collect from customers
> 
> - **DPO** (Days Payable Outstanding) = (AP / COGS) × 365
>   - How long we take to pay suppliers
> 
> - **DIO** (Days Inventory Outstanding) = (Inventory / COGS) × 365
>   - How long inventory sits before sale
> 
> - **CCC** (Cash Conversion Cycle) = DSO + DIO - DPO
>   - Days between paying suppliers and collecting from customers
>   - **Lower is better** (less cash tied up)

In [ ]:
# Calculate working capital metrics
def calculate_wc_metrics(receivables, payables, period_days=365):
    """Calculate key working capital metrics"""
    
    # Total AR and AP
    total_ar = receivables[receivables['status'].isin(['OPEN', 'OVERDUE'])]['amount'].sum()
    total_ap = payables[payables['status'].isin(['OPEN', 'SCHEDULED'])]['amount'].sum()
    
    # Assume annual revenue and COGS (typical for mid-size refinery)
    annual_revenue = 5_000_000_000  # $5B
    annual_cogs = 4_200_000_000     # $4.2B (84% of revenue)
    inventory_value = 300_000_000   # $300M
    
    # Calculate metrics
    dso = (total_ar / annual_revenue) * 365
    dpo = (total_ap / annual_cogs) * 365
    dio = (inventory_value / annual_cogs) * 365
    ccc = dso + dio - dpo
    
    # Net Working Capital
    cash = 150_000_000  # $150M cash
    current_assets = cash + total_ar + inventory_value
    short_term_debt = 200_000_000  # $200M
    current_liabilities = total_ap + short_term_debt
    nwc = current_assets - current_liabilities
    
    return {
        'DSO': dso,
        'DPO': dpo,
        'DIO': dio,
        'CCC': ccc,
        'Total AR': total_ar,
        'Total AP': total_ap,
        'Inventory': inventory_value,
        'Net Working Capital': nwc,
        'Current Ratio': current_assets / current_liabilities
    }

metrics = calculate_wc_metrics(receivables, payables)

print("📊 WORKING CAPITAL METRICS")
print("=" * 50)
print(f"Days Sales Outstanding (DSO):     {metrics['DSO']:.1f} days")
print(f"Days Payable Outstanding (DPO):   {metrics['DPO']:.1f} days")
print(f"Days Inventory Outstanding (DIO): {metrics['DIO']:.1f} days")
print(f"Cash Conversion Cycle (CCC):      {metrics['CCC']:.1f} days")
print("\n" + "=" * 50)
print(f"Total Receivables:    ${metrics['Total AR']/1e6:,.0f}M")
print(f"Total Payables:       ${metrics['Total AP']/1e6:,.0f}M")
print(f"Net Working Capital:  ${metrics['Net Working Capital']/1e6:,.0f}M")
print(f"Current Ratio:        {metrics['Current Ratio']:.2f}")

In [ ]:
# Visualize Cash Conversion Cycle
fig = go.Figure()

# Waterfall for CCC
fig.add_trace(go.Waterfall(
    x=['DSO', 'DIO', 'DPO', 'CCC'],
    y=[metrics['DSO'], metrics['DIO'], -metrics['DPO'], metrics['CCC']],
    measure=['relative', 'relative', 'relative', 'total'],
    text=[f"{metrics['DSO']:.1f}", f"{metrics['DIO']:.1f}", 
          f"-{metrics['DPO']:.1f}", f"{metrics['CCC']:.1f}"],
    textposition='outside',
    connector={'line': {'color': 'rgb(63, 63, 63)'}},
    decreasing={'marker': {'color': 'green'}},
    increasing={'marker': {'color': 'red'}},
    totals={'marker': {'color': 'blue'}}
))

fig.update_layout(
    title='Cash Conversion Cycle Breakdown',
    yaxis_title='Days',
    height=400,
    showlegend=False
)
fig.show()

## 3. Receivables Aging Analysis

### Thinking Trace 🧠

> **Why is AR aging important?**
> 
> - **Current (0-30 days)**: Low risk, expected collections
> - **31-60 days**: Moderate risk, follow-up needed
> - **61-90 days**: High risk, collection efforts required
> - **90+ days**: Very high risk, may need write-off
> 
> AI can predict which invoices are likely to become overdue

In [ ]:
# Calculate aging buckets
today = datetime(2024, 12, 15)

def get_aging_bucket(row):
    if row['status'] == 'PAID':
        return 'PAID'
    days_outstanding = (today - row['invoice_date']).days
    if days_outstanding <= 30:
        return '0-30'
    elif days_outstanding <= 60:
        return '31-60'
    elif days_outstanding <= 90:
        return '61-90'
    else:
        return '90+'

receivables['aging_bucket'] = receivables.apply(get_aging_bucket, axis=1)

# Aging summary
aging_summary = receivables[receivables['status'] != 'PAID'].groupby('aging_bucket').agg({
    'amount': ['sum', 'count'],
    'invoice_id': 'count'
}).round(0)

aging_summary.columns = ['Total Amount', 'Count', 'Invoice Count']
aging_summary = aging_summary.drop('Invoice Count', axis=1)

print("📊 RECEIVABLES AGING ANALYSIS")
print("=" * 50)
print(aging_summary)

In [ ]:
# Visualize aging by customer rating
aging_by_rating = receivables[receivables['status'] != 'PAID'].groupby(
    ['aging_bucket', 'credit_rating']
)['amount'].sum().reset_index()

fig = px.bar(aging_by_rating, x='aging_bucket', y='amount', color='credit_rating',
             title='Receivables Aging by Credit Rating',
             category_orders={'aging_bucket': ['0-30', '31-60', '61-90', '90+'],
                              'credit_rating': ['AAA', 'AA', 'BBB', 'BB', 'B']})
fig.update_layout(yaxis_title='Amount ($)', xaxis_title='Aging Bucket', height=400)
fig.show()

## 4. Late Payment Prediction Model

### Thinking Trace 🧠

> **Predicting late payments allows:**
> 
> 1. **Proactive Collection**: Focus on high-risk invoices early
> 2. **Cash Flow Planning**: Better forecast actual collections
> 3. **Credit Decisions**: Adjust terms for risky customers
> 4. **Reserve Planning**: Provision for bad debts

In [ ]:
# Prepare features for prediction
paid_invoices = receivables[receivables['status'] == 'PAID'].copy()

# Feature engineering
paid_invoices['is_late'] = paid_invoices['days_late'] > 0
paid_invoices['amount_log'] = np.log10(paid_invoices['amount'])
paid_invoices['rating_numeric'] = paid_invoices['credit_rating'].map(
    {'AAA': 5, 'AA': 4, 'BBB': 3, 'BB': 2, 'B': 1}
)
paid_invoices['month'] = paid_invoices['invoice_date'].dt.month
paid_invoices['is_month_end'] = paid_invoices['invoice_date'].dt.day > 25

# Features and target
features = ['amount_log', 'payment_terms', 'rating_numeric', 'month', 'is_month_end']
X = paid_invoices[features]
y = paid_invoices['is_late']

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train model
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Evaluate
y_pred = model.predict(X_test)

print("📊 LATE PAYMENT PREDICTION MODEL")
print("=" * 50)
print("\nClassification Report:")
print(classification_report(y_test, y_pred, target_names=['On-Time', 'Late']))

In [ ]:
# Feature importance
importance_df = pd.DataFrame({
    'Feature': features,
    'Importance': model.feature_importances_
}).sort_values('Importance', ascending=True)

fig = px.bar(importance_df, x='Importance', y='Feature', orientation='h',
             title='Feature Importance for Late Payment Prediction')
fig.update_layout(height=350)
fig.show()

In [ ]:
# Apply model to open invoices
open_invoices = receivables[receivables['status'] == 'OPEN'].copy()

if len(open_invoices) > 0:
    open_invoices['amount_log'] = np.log10(open_invoices['amount'])
    open_invoices['rating_numeric'] = open_invoices['credit_rating'].map(
        {'AAA': 5, 'AA': 4, 'BBB': 3, 'BB': 2, 'B': 1}
    )
    open_invoices['month'] = open_invoices['invoice_date'].dt.month
    open_invoices['is_month_end'] = open_invoices['invoice_date'].dt.day > 25
    
    X_open = open_invoices[features]
    open_invoices['late_probability'] = model.predict_proba(X_open)[:, 1]
    open_invoices['risk_category'] = pd.cut(open_invoices['late_probability'], 
                                             bins=[0, 0.3, 0.6, 1.0],
                                             labels=['Low', 'Medium', 'High'])
    
    print("📊 OPEN INVOICES RISK ASSESSMENT")
    print("=" * 60)
    risk_summary = open_invoices.groupby('risk_category').agg({
        'amount': ['sum', 'count'],
        'late_probability': 'mean'
    }).round(2)
    risk_summary.columns = ['Total Amount', 'Count', 'Avg Late Prob']
    print(risk_summary)
    
    # High risk invoices requiring attention
    high_risk = open_invoices[open_invoices['risk_category'] == 'High'].sort_values(
        'amount', ascending=False
    ).head(10)
    
    print("\n🔴 TOP 10 HIGH-RISK INVOICES:")
    print(high_risk[['invoice_id', 'customer', 'amount', 'due_date', 'late_probability']].to_string(index=False))

## 5. Payment Optimization Strategy

### Thinking Trace 🧠

> **When should we pay early for discounts?**
> 
> Compare discount APR to cost of capital:
> - Discount APR = (Discount % / (100 - Discount %)) × (365 / (Terms - Discount Days))
> - Example: 2% discount for paying in 10 days vs 30 days
>   - APR = (2/98) × (365/20) = 37.2%
> - If our cost of capital < 37.2%, take the discount!

In [ ]:
def calculate_discount_apr(discount_pct, discount_days, payment_terms):
    """Calculate APR equivalent of early payment discount"""
    if discount_pct == 0 or payment_terms <= discount_days:
        return 0
    
    # APR = (discount / (100 - discount)) × (365 / days saved)
    days_saved = payment_terms - discount_days
    apr = (discount_pct / (100 - discount_pct)) * (365 / days_saved) * 100
    return apr

# Analyze payables with discounts
payables_with_discount = payables[payables['early_discount_pct'] > 0].copy()

payables_with_discount['discount_apr'] = payables_with_discount.apply(
    lambda r: calculate_discount_apr(r['early_discount_pct'], r['early_discount_days'], r['payment_terms']),
    axis=1
)

payables_with_discount['discount_value'] = (
    payables_with_discount['amount'] * payables_with_discount['early_discount_pct'] / 100
)

# Cost of capital threshold
cost_of_capital = 8.0  # 8% annual

payables_with_discount['take_discount'] = payables_with_discount['discount_apr'] > cost_of_capital

print("📊 EARLY PAYMENT DISCOUNT ANALYSIS")
print("=" * 60)
print(f"Cost of Capital: {cost_of_capital}%")
print(f"\nPayables with Discounts: {len(payables_with_discount)}")
print(f"Total Amount: ${payables_with_discount['amount'].sum()/1e6:,.1f}M")
print(f"\nDiscounts Worth Taking: {payables_with_discount['take_discount'].sum()}")
print(f"Potential Savings: ${payables_with_discount[payables_with_discount['take_discount']]['discount_value'].sum():,.0f}")

In [ ]:
# Visualize discount opportunities
fig = px.scatter(payables_with_discount, x='amount', y='discount_apr',
                 size='discount_value', color='take_discount',
                 hover_data=['supplier', 'category'],
                 title='Early Payment Discount Opportunities',
                 labels={'amount': 'Invoice Amount ($)', 'discount_apr': 'Effective APR (%)'})

fig.add_hline(y=cost_of_capital, line_dash="dash", line_color="red",
              annotation_text=f"Cost of Capital ({cost_of_capital}%)")
fig.update_layout(height=400)
fig.show()

## 6. Working Capital Forecast

### Thinking Trace 🧠

> **What drives working capital forecasts?**
> 
> 1. **Sales Forecast** → Expected receivables
> 2. **Purchase Plan** → Expected payables
> 3. **Production Schedule** → Inventory levels
> 4. **Collection Patterns** → Cash timing
> 5. **Seasonality** → Demand variations

In [ ]:
# Generate 12-month working capital forecast
def forecast_working_capital(months=12, seed=42):
    np.random.seed(seed)
    
    base_date = datetime(2025, 1, 1)
    forecasts = []
    
    # Base values
    base_ar = 400_000_000
    base_ap = 350_000_000
    base_inventory = 300_000_000
    base_cash = 150_000_000
    
    for m in range(months):
        date = base_date + timedelta(days=30*m)
        
        # Seasonal factors
        month = date.month
        seasonal_factor = 1 + 0.15 * np.sin(2 * np.pi * (month - 3) / 12)  # Peak in summer
        
        # Forecast with seasonal variation + noise
        ar = base_ar * seasonal_factor * (1 + np.random.uniform(-0.05, 0.05))
        ap = base_ap * seasonal_factor * (1 + np.random.uniform(-0.05, 0.05))
        inventory = base_inventory * (1 + np.random.uniform(-0.1, 0.1))
        cash = base_cash * (1 + np.random.uniform(-0.2, 0.2))
        
        # Calculate metrics
        nwc = (cash + ar + inventory) - (ap + 200_000_000)  # Short-term debt
        
        forecasts.append({
            'date': date,
            'month': date.strftime('%b'),
            'AR': ar,
            'AP': ap,
            'Inventory': inventory,
            'Cash': cash,
            'NWC': nwc
        })
    
    return pd.DataFrame(forecasts)

forecast = forecast_working_capital(12, seed=42)

print("📊 12-MONTH WORKING CAPITAL FORECAST")
print("=" * 70)
display(forecast[['month', 'AR', 'AP', 'Inventory', 'Cash', 'NWC']].round(-6))

In [ ]:
# Visualize forecast
fig = make_subplots(rows=2, cols=1,
    subplot_titles=('Working Capital Components', 'Net Working Capital Trend'),
    vertical_spacing=0.15)

# Components
for component in ['AR', 'AP', 'Inventory', 'Cash']:
    fig.add_trace(go.Scatter(x=forecast['month'], y=forecast[component]/1e6,
                             name=component, mode='lines+markers'), row=1, col=1)

# NWC trend
fig.add_trace(go.Scatter(x=forecast['month'], y=forecast['NWC']/1e6,
                         name='NWC', mode='lines+markers', fill='tozeroy',
                         line=dict(color='blue', width=3)), row=2, col=1)

fig.update_yaxes(title_text='$ Million', row=1, col=1)
fig.update_yaxes(title_text='$ Million', row=2, col=1)
fig.update_layout(height=600, title_text='Working Capital Forecast')
fig.show()

## 7. Executive Dashboard

In [ ]:
# Create executive dashboard
fig = make_subplots(
    rows=2, cols=3,
    subplot_titles=(
        'Cash Conversion Cycle', 'AR Aging Distribution', 'Collection Performance',
        'Net Working Capital', 'Payables by Category', 'DSO Trend'
    ),
    specs=[[{"type": "indicator"}, {"type": "pie"}, {"type": "bar"}],
           [{"type": "indicator"}, {"type": "pie"}, {"type": "scatter"}]]
)

# 1. CCC gauge
fig.add_trace(go.Indicator(
    mode="gauge+number",
    value=metrics['CCC'],
    title={'text': "CCC (Days)"},
    gauge={'axis': {'range': [0, 100]},
           'bar': {'color': "darkblue"},
           'steps': [{'range': [0, 30], 'color': 'green'},
                     {'range': [30, 60], 'color': 'yellow'},
                     {'range': [60, 100], 'color': 'red'}]}
), row=1, col=1)

# 2. AR aging pie
aging_data = receivables[receivables['status'] != 'PAID'].groupby('aging_bucket')['amount'].sum()
colors = {'0-30': 'green', '31-60': 'yellow', '61-90': 'orange', '90+': 'red'}
fig.add_trace(go.Pie(labels=aging_data.index, values=aging_data.values,
                     marker_colors=[colors.get(b, 'gray') for b in aging_data.index]), 
              row=1, col=2)

# 3. Collection performance by rating
collection_perf = receivables[receivables['status'] == 'PAID'].groupby('credit_rating').agg({
    'days_late': 'mean'
}).reset_index()
fig.add_trace(go.Bar(x=collection_perf['credit_rating'], y=collection_perf['days_late'],
                     marker_color=['green' if d < 5 else 'orange' if d < 15 else 'red' 
                                   for d in collection_perf['days_late']]), 
              row=1, col=3)

# 4. NWC gauge
fig.add_trace(go.Indicator(
    mode="number+delta",
    value=metrics['Net Working Capital']/1e6,
    title={'text': "NWC ($M)"},
    delta={'reference': 200, 'relative': False},
    number={'suffix': 'M'}
), row=2, col=1)

# 5. AP by category
ap_by_cat = payables.groupby('category')['amount'].sum()
fig.add_trace(go.Pie(labels=ap_by_cat.index, values=ap_by_cat.values, hole=0.4), row=2, col=2)

# 6. DSO trend (simulated)
months = pd.date_range(end=datetime.now(), periods=12, freq='M')
dso_trend = 35 + np.random.randn(12).cumsum() * 2
fig.add_trace(go.Scatter(x=months, y=dso_trend, mode='lines+markers', name='DSO'), row=2, col=3)
fig.add_hline(y=30, line_dash="dash", line_color="green", row=2, col=3, annotation_text="Target")

fig.update_layout(height=600, title_text='Working Capital Management Dashboard', showlegend=False)
fig.show()

## 8. Key Takeaways

### AI Applications for Working Capital

| Application | Model/Technique | Benefit |
|------------|-----------------|--------|
| **Late Payment Prediction** | Random Forest, XGBoost | Focus collection efforts |
| **Cash Flow Forecasting** | Prophet, LSTM | Better liquidity planning |
| **Discount Optimization** | Rules + Optimization | Capture savings opportunities |
| **Inventory Optimization** | Demand forecasting | Reduce carrying costs |
| **Customer Segmentation** | Clustering | Tailored credit terms |

### Optimization Opportunities

| Lever | Current | Target | Impact |
|-------|---------|--------|--------|
| DSO | 35 days | 30 days | +$68M cash |
| DPO | 28 days | 35 days | +$80M cash |
| Early Pay Discounts | 20% | 80% | +$500K savings |
| Bad Debt | 1.5% | 0.8% | +$3.5M |

### Implementation Roadmap

1. **Phase 1**: Data integration (AR/AP/Inventory)
2. **Phase 2**: Predictive models deployment
3. **Phase 3**: Process automation
4. **Phase 4**: Optimization algorithms

---

*Author: Ozgur Guler (ozgur.guler1@gmail.com)*